In [ ]:
import os
import struct
import gc
import numpy as np
from keras.utils import np_utils
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
%matplotlib inline

# 1 . Create Dataset

In [ ]:
from keras.datasets import cifar100
(X_train, y_train), (X_test, y_test) = cifar100.load_data() 
X_train = X_train / 255
X_test  = X_test / 255

In [1]:
cv = 10

In [ ]:
def gen_add_noise_50(X_train,y_train):
    noise_random = np.random.rand(50000,32,32,3)
    noise_labels = np.full((50000,1), 100, dtype=int)
    X_train_60 = np.concatenate((X_train, noise_random), axis = 0 )
    y_train_60 = np.concatenate((y_train, noise_labels), axis = 0 )
    
    return X_train_60,y_train_60

In [ ]:
def gen_add_noise_5(X_train,y_train):
    noise_random = np.random.rand(500,32,32,3)
    noise_labels = np.full((500,1), 100, dtype=int)
    X_train_6 = np.concatenate((X_train, noise_random), axis = 0 )
    y_train_6 = np.concatenate((y_train, noise_labels), axis = 0 )
    
    return X_train_6,y_train_6

In [ ]:
def shuffle_me(X_train,y_train):
    X_sparse_train = coo_matrix(X_train)
    X_train, X_sparse_train, y_train = shuffle(X_train, X_sparse_train, y_train, random_state=42)
    return X_train,y_train

In [ ]:
def keras_style_ds(X_train, X_test, y_train, y_test,original):
# keras - input

    if(original == 1):
        num_classes = 100               
    else:
        num_classes = 101
        
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test  = np_utils.to_categorical(y_test, num_classes)
    
    return X_train, X_test, y_train, y_test, num_classes

# 2 . Build Model -- keras model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta



In [ ]:
def model_factory():
    K.clear_session()
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', 
                     data_format='channels_last',
                     input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
    
    return model


In [ ]:
# model = model_factory()
# model.summary()

# 3. Training & Test

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(cv, random_state=None, shuffle=False)

In [ ]:
# Train no random noise added
hs_mnist_no_scores = []
hs_mnist_no_history= []


for i in range(30):
    
    X_train_no,y_train_no = X_train,y_train
    X_train_no,y_train_no = shuffle_me(X_train_no,y_train_no)
    X_train_no, X_test_no, y_train_no, y_test_no, num_classes = keras_style_ds(X_train_no, X_test, 
                                                                               y_train_no, y_test, 1)

    model_no = model_factory()

    for train_index, test_index in kf.split(X_train_no):
        print("TRAIN:", len(train_index), "TEST:", len(test_index))

        X_train_cv, X_test_cv = X_train_no[train_index], X_train_no[test_index]
        y_train_cv, y_test_cv = y_train_no[train_index], y_train_no[test_index]

        history = model_no.fit(X_train_cv, y_train_cv, epochs=12, batch_size=32, 
                               validation_data = (X_test_cv, y_test_cv),verbose=1)
        hs_mnist_no_history.append(history)  

    test_no_score = model_no.evaluate(X_test_no, y_test_no, verbose=1)
    hs_mnist_no_scores.append(test_no_score)

    model_no.save("./models/CIFAR100_dnn_10fold_0426_no_%d.h5" % (i+1))

    print()
    print("-------------%d  time training and testing -----------------------------"% ( i+1) )
    print("CIFAR100 test sets -> Loss: %f" % (test_no_score[0]))
    print("CIFAR100 test sets -> Accuracy: %.2f%%" % (test_no_score[1]*100))
    
    gc.collect()

In [ ]:
hs_mnist_no_scores

In [ ]:
# Train with 6000 random noise added
hs_mnist_6_scores = []
hs_mnist_6_history= []


for j in range(30):
    
    X_train_6,y_train_6 = gen_add_noise_5(X_train,y_train)
    X_train_6,y_train_6 = shuffle_me(X_train_6,y_train_6)
    X_train_6, X_test_6, y_train_6, y_test_6 , num_classes = keras_style_ds(X_train_6, X_test, 
                                                                            y_train_6, y_test, 0)
    
    model_6 = model_factory()

    for train_index, test_index in kf.split(X_train_6):
        print("TRAIN:", len(train_index), "TEST:", len(test_index))

        X_train_6_cv, X_test_6_cv = X_train_6[train_index], X_train_6[test_index]
        y_train_6_cv, y_test_6_cv = y_train_6[train_index], y_train_6[test_index]

#         X_train_6_cv, X_test_6_cv, y_train_6_cv, y_test_6_cv = keras_style_ds(X_train_cv, X_test_cv, y_train_cv, y_test_cv)
        history_6 = model_6.fit(X_train_6_cv, y_train_6_cv, epochs=12, batch_size=32, 
                                validation_data = (X_test_6_cv, y_test_6_cv),verbose=1)
        hs_mnist_6_history.append(history_6)  

    test_6_score = model_6.evaluate(X_test_6, y_test_6, verbose=1)
    hs_mnist_6_scores.append(test_6_score)
    
    model_6.save("./models/CIFAR100_dnn_10fold_0426_5k_%d.h5" % (j+1))

    print()
    print("-------------%d  time training and testing -----------------------------"% ( j+1) )
    print("CIFAR100 5 test sets -> Loss: %f" % (test_6_score[0]))
    print("CIFAR100 5 test sets -> Accuracy: %.2f%%" % (test_6_score[1]*100))
    
    gc.collect()

In [ ]:
hs_mnist_6_scores

In [ ]:
# Train with 60,000 random noise added
hs_mnist_60_scores = []
hs_mnist_60_history= []


for h in range(30):
    
    X_train_60,y_train_60 = gen_add_noise_50(X_train,y_train)
    X_train_60,y_train_60 = shuffle_me(X_train_60,y_train_60)
    X_train_60, X_test_60, y_train_60, y_test_60 , num_classes = keras_style_ds(X_train_60, X_test, 
                                                                                y_train_60, y_test, 0)
    
    model_60 = model_factory()

    for train_index, test_index in kf.split(X_train_6):
        print("TRAIN:", len(train_index), "TEST:", len(test_index))

        X_train_60_cv, X_test_60_cv = X_train_60[train_index], X_train_60[test_index]
        y_train_60_cv, y_test_60_cv = y_train_60[train_index], y_train_60[test_index]

        history_60 = model_60.fit(X_train_60_cv, y_train_60_cv, epochs=12, batch_size=32, 
                                  validation_data = (X_test_60_cv, y_test_60_cv),verbose=1)
        
        hs_mnist_60_history.append(history_60)  

    test_60_score = model_60.evaluate(X_test_60, y_test_60, verbose=1)
    hs_mnist_60_scores.append(test_60_score)

    model_60.save("./models/CIFAR100_dnn_10fold_0426_50k_%d.h5" % (h+1))

    print()
    print("-------------%d  time training and testing -----------------------------"% ( h+1) )
    print("CIFAR100 50k test sets -> Loss: %f" % (test_60_score[0]))
    print("CIFAR100 50k test sets -> Accuracy: %.2f%%" % (test_60_score[1]*100))
    
    gc.collect()

In [ ]:
hs_mnist_60_scores